In [1]:
from gensim.models import KeyedVectors
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd
from datetime import datetime
from tqdm import tqdm
from polyglot.mapping import Embedding

In [2]:
neighbour_list = [5, 10, 25, 50, 75, 100]
embedding_list = ['polyglot', 'cc']
embedding_path = '../data/pretrained_embeddings/'
# embedding_path = '../data/pretrained_embeddings/polyglot/embeddings2/'

In [3]:
def get_embedding_file_name(language, file_type):
    lang_code = ''
    file_name = ''
    if (language == "Russian"):
        lang_code += 'ru'
    elif (language == "Turkish"):
        lang_code += 'tr'
    elif (language == "German"):
        lang_code += 'de'
    elif (language == "Japanese"):
        lang_code += 'ja'
    elif (language == "Spanish"):
        lang_code += 'es'
    elif (language == "Polish"):
        lang_code += 'pl'
    elif (language == "Italian"):
        lang_code += 'it'
    elif (language == "Catalan"):
        lang_code += 'ca'
    elif (language == "Korean"):
        lang_code += 'ko'
    elif (language == "French"):
        lang_code += 'fr'
    elif (language == "Chinese" or language == "Chinese (Simplified)"):
        lang_code += 'zh'
    elif (language == "Portuguese"):
        lang_code += 'pt'
    elif (language == "Swedish"):
        lang_code += 'sv'
    elif (language == "Greek"):
        lang_code += 'el'
    elif (language == "Thai"):
        lang_code += 'th'
    elif (language == "Dutch"):
        lang_code += 'nl'
    elif (language == "English" and file_type == 'cc'):
        lang_code += 'en.300'
    elif (language == "English" and file_type == 'polyglot'):
        lang_code += 'en'
        
    if (file_type == 'cc'):
        file_name = embedding_path + file_type + '.' + lang_code + '.vec'
    elif (file_type == 'polyglot'):
        file_name = embedding_path + file_type + '/embeddings2/' + lang_code + '/embeddings_pkl.tar.bz2'
    return file_name

def load_embedding_vector(language, file_type):
    file_name = get_embedding_file_name(language, file_type)
    print('Loading file: ' + file_name)
    if (file_type == 'cc'):
        print('Loading fasttext')
        vec = KeyedVectors.load_word2vec_format(file_name, binary=False, unicode_errors='ignore')  # C text format
    elif (file_type == 'polyglot'):
        print('Loading polyglot')
        vec = Embedding.load(file_name)
        vec = vec.normalize_words()
    return vec

In [4]:
def proper_case(word):
    return word[0].upper() + word[1:]

def get_neigbours(word, embeddings, vector_type, size, sorting_type = 'desc'):
    res1 = []
    res2 = []
    
    try:
        w = embeddings[word]
    except KeyError:
        if (vector_type == 'polyglot'):
            word = proper_case(word)
            try:
                w = embeddings[word]
            except KeyError:
                return -10, -10
        else:
            return -10, -10
    if (vector_type == 'polyglot'):
        n = embeddings.nearest_neighbors(word, top_k = size)
        n_v = [embeddings[word] for word in n]
        n_s = cosine_similarity([embeddings[word]], n_v)
        for index, item in enumerate(n_s[0]):
            res1.append(n[index])
            res2.append({'word': n[index], 'similarity': str(item)})
    elif (vector_type == 'cc'):
        n = embeddings.similar_by_vector(embeddings[word], topn = size + 1)
        for item in n[1:]:
            res1.append(item[0])
            res2.append({'word': item[0], 'similarity': str(item[1])})
    if (sorting_type == 'asc'):
        res1 = res1[::-1]
        res2 = res2[::-1]
    return res1, res2

In [5]:
# source = '../data/ms_final_with_polyglot_2.csv'
# pd_dataset = pd.read_csv(source)
# pd_dataset = pd_dataset.sort_values(by = 'language', ascending = True)
# pd_dataset.head()

In [6]:
# test = load_embedding_vector("English", "polyglot")

In [7]:
# test = test.normalize_words()
# n = test.nearest_neighbors("hello")
# print(n)
# n_v = [test[word] for word in n]
# n_s = cosine_similarity([test["hello"]], n_v)

# for i, item in enumerate(n_s[0]):
#     print(item)
# res1, res2 = get_neigbours("hello", test, "polyglot", 10, 'desc')
# print(res1)
# print(res2)

In [8]:
# test = load_embedding_vector("English", "cc")

In [9]:
# n = test.similar_by_vector(test["hello"])
# def get_formatted_neighbours(neighbours, vector_type):
#     res1 = []
#     res2 = []
#     return res1, res2

In [10]:
source = '../data/fasttext_experiments.csv'
pd_dataset = pd.read_csv(source)
pd_dataset = pd_dataset.sort_values(by = 'language', ascending = True)
pd_dataset.head()

,c,i,id,l1_c,l1_i,language,type,code,l2_sim_cc_5,l2_sim_cc_10,...,l2_sim_cc_50,l2_sim_cc_75,l2_sim_cc_100,l1_sim_cc_5,l1_sim_cc_10,l1_sim_cc_25,l1_sim_cc_50,l1_sim_cc_75,l1_sim_cc_100,case_id
0,plans,projects,person_2799,plans,projectes,Catalan,RN,ca,0.315914,0.289572,...,0.246446,0.235596,0.228922,0.534060,0.509746,0.513301,0.508249,0.497365,0.494275,case_0001
221,stars,starts,person_2127,estrelles,comença,Catalan,RN,ca,0.068227,0.068642,...,0.062946,0.059115,0.056152,0.168406,0.167430,0.177438,0.174349,0.173036,0.175587,case_0222
220,time,hours,person_2127,temps,hores,Catalan,RN,ca,0.464017,0.407275,...,0.373413,0.359142,0.354130,0.380932,0.359519,0.346928,0.349675,0.352660,0.349513,case_0221
219,coming,following,person_2127,venint,següents,Catalan,RJ,ca,0.183514,0.166062,...,0.170379,0.165272,0.161477,0.182136,0.181615,0.211532,0.218019,0.222701,0.225519,case_0220
218,equipment,material,person_2127,equip,material,Catalan,RN,ca,0.279342,0.251406,...,0.225062,0.223828,0.219889,0.352026,0.340105,0.323130,0.316861,0.310028,0.312223,case_0219


In [11]:
fasttext_result_set = {}
fasttext_result_set_sim = {}
def get_english_data(dataframe, vector_type, rset1, rset2):
    vec_type_start = datetime.now()
    print('Loading embeddings')
    embeddings = load_embedding_vector("English", vector_type)
    print('Embeddings for english, ' + vector_type + ' vectors loaded in ' + str(datetime.now() - vec_type_start))
    for ind, row in tqdm(dataframe.iterrows(), total=dataframe.shape[0]):
        c = row['c']
        i = row['i']
        case_id = row['case_id']
        c_n, c_n_d = get_neigbours(c, embeddings, vector_type, 100, 'desc')
        i_n, i_n_d = get_neigbours(i, embeddings, vector_type, 100, 'desc')
        
        if (case_id not in rset1):
            rset1[case_id] = {}
        if (case_id not in rset2):
            rset2[case_id] = {}
        
        rset1[case_id]['i_nn'] = i_n
        rset1[case_id]['c_nn'] = c_n
        rset2[case_id]['i_nn'] = i_n_d
        rset2[case_id]['c_nn'] = c_n_d


In [12]:
# get_english_data(pd_dataset, "cc", fasttext_result_set, fasttext_result_set_sim)

In [13]:
import json

In [14]:
# with open('fasttext_result_set_english.json', 'w') as fp:
#     json.dump(fasttext_result_set, fp)
# with open('fasttext_result_set_sim_english.json', 'w') as fp:
#     json.dump(fasttext_result_set_sim, fp)


In [15]:
language_list = pd_dataset['language'].unique()

def get_l1_data(dataframe, language_list, vector_type, rset1, rset2):
    current_lang = ''
    for language in language_list:
        language_start = datetime.now()
        vec_type_start = datetime.now()
        print('Loading embeddings')
        embeddings = load_embedding_vector(language, vector_type)
        print('Embeddings for ' + language + ' , ' + vector_type + ' vectors loaded in ' + str(datetime.now() - vec_type_start))
        dset = dataframe[dataframe['language'] == language]
        for ind, row in tqdm(dset.iterrows(), total=dset.shape[0]):
            c = row['l1_c']
            i = row['l1_i']
            case_id = row['case_id']
            c_n, c_n_d = get_neigbours(c, embeddings, vector_type, 100, 'desc')
            i_n, i_n_d = get_neigbours(i, embeddings, vector_type, 100, 'desc')

            if (case_id not in rset1):
                rset1[case_id] = {}
            if (case_id not in rset2):
                rset2[case_id] = {}

            rset1[case_id]['l1_i_nn'] = i_n
            rset1[case_id]['l1_c_nn'] = c_n
            rset2[case_id]['l1_i_nn'] = i_n_d
            rset2[case_id]['l1_c_nn'] = c_n_d

        print('Finished run for language: ' + language + ' in ' + str(datetime.now() - language_start))

In [16]:
# get_l1_data(pd_dataset, language_list, "cc", fasttext_result_set, fasttext_result_set_sim)

In [17]:
# with open('fasttext_result_set.json', 'w') as fp:
#     json.dump(fasttext_result_set, fp)
# with open('fasttext_result_set_sim.json', 'w') as fp:
#     json.dump(fasttext_result_set_sim, fp)


In [18]:
source = '../data/polyglot_experiment.csv'
pd2_dataset = pd.read_csv(source)
pd2_dataset = pd2_dataset.sort_values(by = 'language', ascending = True)
pd2_dataset.head()

,c,i,id,l1_c,l1_i,language,type,code,l2_sim_cc_5,l2_sim_cc_10,...,l2_sim_polyglot_50,l2_sim_polyglot_75,l2_sim_polyglot_100,l1_sim_polyglot_5,l1_sim_polyglot_10,l1_sim_polyglot_25,l1_sim_polyglot_50,l1_sim_polyglot_75,l1_sim_polyglot_100,case_id
0,plans,projects,person_2799,plans,projectes,Catalan,RN,ca,0.315914,0.289572,...,0.642290,0.636711,0.636448,0.706547,0.719866,0.709139,0.681242,0.676333,0.667461,case_0001
211,especially,specially,person_2421,especialment,especialment,Catalan,RY,ca,0.342199,0.336672,...,0.513900,0.486702,0.479133,0.828464,0.793318,0.749138,0.716610,0.689578,0.673222,case_0009
210,waste,lose,person_2421,malgastar,perdre,Catalan,RV,ca,0.197119,0.175892,...,0.098449,0.101967,0.105505,0.631330,0.636749,0.636339,0.637038,0.635354,0.634739,case_0002
209,ending,end,person_1465,final,final,Catalan,RN,ca,0.591592,0.500977,...,0.409819,0.411196,0.404940,0.784284,0.765748,0.735284,0.709147,0.690993,0.676256,case_0003
208,bored,boring,person_1642,avorrit,avorrit,Catalan,RJ,ca,0.513438,0.502850,...,0.627174,0.611835,0.597600,0.733787,0.727427,0.721897,0.708325,0.698112,0.690430,case_0026


In [19]:
poly_result_set = {}
poly_result_set_sim = {}

In [20]:
poly_result_set = {}
poly_result_set_sim = {}
fasttext_result_set = {}
fasttext_result_set_sim = {}
language_list = pd_dataset['language'].unique()
language_list2 = pd2_dataset['language'].unique()

get_english_data(pd_dataset, "cc", fasttext_result_set, fasttext_result_set_sim)
with open('fasttext_result_set_eng.json', 'w') as fp:
    json.dump(fasttext_result_set, fp)
with open('fasttext_result_set_sim_eng.json', 'w') as fp:
    json.dump(fasttext_result_set_sim, fp)
get_l1_data(pd_dataset, language_list, "cc", fasttext_result_set, fasttext_result_set_sim)
with open('fasttext_result_set.json', 'w') as fp:
    json.dump(fasttext_result_set, fp)
with open('fasttext_result_set_sim.json', 'w') as fp:
    json.dump(fasttext_result_set_sim, fp)
get_english_data(pd2_dataset, "polyglot", poly_result_set, poly_result_set_sim)
with open('poly_result_set_eng.json', 'w') as fp:
    json.dump(poly_result_set, fp)
with open('poly_result_set_sim_eng.json', 'w') as fp:
    json.dump(poly_result_set_sim, fp)
get_l1_data(pd2_dataset, language_list2, "polyglot", poly_result_set, poly_result_set_sim)
with open('poly_result_set.json', 'w') as fp:
    json.dump(poly_result_set, fp)
with open('poly_result_set_sim.json', 'w') as fp:
    json.dump(poly_result_set_sim, fp)

Loading embeddings


  0%|          | 0/4937 [00:00<?, ?it/s]

Loading file: ../data/pretrained_embeddings/cc.en.300.vec
Loading fasttext
Embeddings for english, cc vectors loaded in 0:07:09.103982


100%|██████████| 4937/4937 [11:48<00:00,  6.97it/s]


Loading embeddings
Loading file: ../data/pretrained_embeddings/cc.ca.vec
Loading fasttext


...

Finished run for language: Thai in 0:00:03.644029
Loading embeddings
Loading file: ../data/pretrained_embeddings/polyglot/embeddings2/tr/embeddings_pkl.tar.bz2
Loading polyglot


  1%|          | 2/226 [00:00<00:13, 17.03it/s]

Embeddings for Turkish , polyglot vectors loaded in 0:00:02.302487


100%|██████████| 226/226 [00:09<00:00, 22.66it/s]


Finished run for language: Turkish in 0:00:12.282282


In [25]:
# for key in fasttext_result_set:
#     print(key + ':' + str(len(fasttext_result_set[key])))
print(fasttext_result_set['case_3309'].keys())
print(fasttext_result_set_sim['case_3309'].keys())

dict_keys(['i_nn', 'c_nn', 'l1_i_nn', 'l1_c_nn'])
dict_keys(['i_nn', 'c_nn', 'l1_i_nn', 'l1_c_nn'])


In [26]:
# fasttext_result_set

{'case_0001': {},
 'i_nn': ['gives',
  'giving',
  'gave',
  'Give',
  'provide',
  'get',
  'gve',
  'want',
  'take',
  'ask',
  'let',
  'try',
  'given',
  'bring',
  'chance',
  'allow',
  'help',
  '.Give',
  'make',
  'need',
  'offer',
  'lend',
  'send',
  'you',
  'tell',
  'put',
  'gving',
  'givng',
  '.give',
  'it.Give',
  'time.Give',
  'opportunity',
  'gice',
  'me.Give',
  'wanted',
  'giveing',
  'encourage',
  'GIve',
  'you.Give',
  'them.Give',
  'beg',
  'invite',
  'andgive',
  'them',
  'lets',
  'leave',
  'givs',
  'giv',
  'givin',
  'implore',
  'come',
  'receive',
  'convince',
  'givem',
  'wants',
  'remind',
  'Gave',
  'bestow',
  "'ll",
  'Gives',
  'add',
  'oppurtunity',
  'opporunity',
  'seek',
  'hope',
  'entice',
  'deprive',
  'togive',
  'advise',
  'oppotunity',
  'enable',
  'deliver',
  'guve',
  'persuade',
  'will',
  'deserve',
  'gladly',
  'nudge',
  'gimme',
  'withhold',
  'givee',
  'impart',
  'look',
  'obtain',
  'owe',
  'abl